In [1]:
# 데이터 불러오기
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Concatenate
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

from tensorflow.keras.models import Model

DATA_PATH = 'C:/Work/1stDL/Roadbook_DeepLearning-master/TF_2.12v/code/clothes_classification/csv_data/colorinfo'

train_df = pd.read_csv(DATA_PATH + '/train_color.csv')
val_df = pd.read_csv(DATA_PATH + '/val_color.csv')
test_df = pd.read_csv(DATA_PATH + '/test_color.csv')

# Colab에서 사용한다면, 다음 코드 주석을 풀고, 실행시킵니다.
# train_df['image'] = train_df['image'].apply(lambda x: str(x).replace('//', '/'))
# val_df['image'] = val_df['image'].apply(lambda x: str(x).replace('//', '/'))
# test_df['image'] = test_df['image'].apply(lambda x: str(x).replace('//', '/'))

train_df.head()

,image,black,blue,brown,green,red,white,dress,shirt,pants,shorts,shoes,color
0,./clothes_dataset\green_shoes\f1f33bed259f4b38...,0,0,0,1,0,0,0,0,0,0,1,3
1,./clothes_dataset\brown_pants\8a797ffb710eefe3...,0,0,1,0,0,0,0,0,1,0,0,2
2,./clothes_dataset\white_dress\ef86bf5eee72dbe8...,0,0,0,0,0,1,1,0,0,0,0,5
3,./clothes_dataset\black_shoes\ff7f558959757ab7...,1,0,0,0,0,0,0,0,0,0,1,0
4,./clothes_dataset\blue_pants\b354ab5371b90d5eb...,0,1,0,0,0,0,0,0,1,0,0,1


In [13]:
# 제네레이터 정의하기
def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0:
        return (num_samples // batch_size) + 1
    else:
        return num_samples // batch_size
    
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, batch_size = 32, target_size = (112, 112),
                shuffle = True, 
                training = True):
        self.len_df = len(df)
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        self.training = training

        if training:
            self.class_col = ['black', 'blue', 'brown', 'green', 'red', 'white',
                              'dress', 'shirt', 'pants', 'shorts', 'shoes']
        else:
            self.class_col = None

        # 제네레이터를 통해 이미지를 불러옵니다.
        self.generator = ImageDataGenerator(rescale=1./255)
        self.df_generator = self.generator.flow_from_dataframe(dataframe = df,
                                                               directory = 'C:/Work/1stDL/Roadbook_DeepLearning-master/TF_2.12v/code/clothes_classification',
                                                               x_col = 'image',
                                                               y_col = self.class_col if training else None,
                                                               target_size = self.target_size,
                                                               color_mode = 'rgb',
                                                               class_mode = 'raw' if training else None,
                                                               batch_size = self.batch_size,
                                                               shuffle = True,
                                                               seed = 42)
        self.colors_df = df['color']
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.len_df) / self.batch_size)
    
    # 데이터를 섞습니다.
    def on_epoch_end(self):
        self.indexes = np.arange(self.len_df)
        if self.shuffle:
            np.random.shuffle(self.indexes)

    # ([이미지 데이터, 색 정보], 레이블)을 반환합니다.
    # 이미지는 미리 정의한 제네레이터를 통해,
    # 색 정보는 __data_generation 메소드를 활용합니다.
    def __getitem__(self, index):
        start = index * self.batch_size
        end = (index + 1) * self.batch_size if (index + 1) * self.batch_size < self.len_df else (self.len_df + 1)
    
        indexes = self.indexes[start:end]
        colors = self.__data_generation(indexes)

        if self.training:
                images, labels = self.df_generator.__getitem__(index)
                return {"images": images, "colors": colors}, labels
        else:
            images = self.df_generator.__getitem__(index)
            return {"images": images, "colors": colors}
        
    def __data_generation(self, indexes):
        colors = np.array([self.colors_df[k] for k in indexes])

        return colors


In [14]:
train_datagen = DataGenerator(train_df, 
                              batch_size = 32,
                              target_size = (112, 112),
                              shuffle = True,
                              training = True)
val_datagen = DataGenerator(val_df,
                            batch_size = 32,
                            target_size = (112, 112),
                            shuffle = True,
                            training = True)

Found 5578 validated image filenames.
Found 2391 validated image filenames.


In [15]:
# 모델 구성하기

def get_model():
    img_input = Input(shape=(112, 112, 3), name='images')
    color_input = Input(shape=[1], name='colors')

    x = Conv2D(32, (3, 3), padding = 'same', activation = 'relu')(img_input)
    x = MaxPooling2D((3, 3), strides = 2)(x)
    x = Conv2D(64, (3, 3), padding = 'same', activation = 'relu')(x)
    x = MaxPooling2D((3, 3), strides = 2)(x)
    x = Conv2D(64, (3, 3), padding = 'same', activation = 'relu')(x)
    x = MaxPooling2D((3, 3), strides = 2)(x)
    x = GlobalAveragePooling2D()(x)

    # 색 데이터를 병합합니다.
    color_concat = Concatenate()([x, color_input])

    x = Dense(64, activation = 'relu')(color_concat)
    x = Dense(11, activation = 'sigmoid')(x)

    # 다중 입력이기 때문에,
    # inputs 인자에 리스트 형태로 입력 데이터를 전달합니다.
    model = Model(inputs={'images': img_input, 'colors': color_input}, outputs=x)

    model.compile(optimizer = 'adam',
                  loss = 'binary_crossentropy',
                  metrics = ['binary_accuracy'])
    
    return model

model = get_model()
print('model ready!')

model ready!


In [16]:
history = model.fit(train_datagen,
                    validation_data = val_datagen,
                    epochs = 10,
                    batch_size = 32)

C:\Users\human\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 62s 344ms/step - binary_accuracy: 0.7833 - loss: 0.4782 - val_binary_accuracy: 0.8637 - val_loss: 0.3119
Epoch 2/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 42s 242ms/step - binary_accuracy: 0.8723 - loss: 0.3028 - val_binary_accuracy: 0.8906 - val_loss: 0.2619
Epoch 3/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 42s 242ms/step - binary_accuracy: 0.8946 - loss: 0.2557 - val_binary_accuracy: 0.9023 - val_loss: 0.2378
Epoch 4/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 42s 239ms/step - binary_accuracy: 0.9046 - loss: 0.2351 - val_binary_accuracy: 0.9067 - val_loss: 0.2251
Epoch 5/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 42s 240ms/step - binary_accuracy: 0.9106 - loss: 0.2196 - val_binary_accuracy: 0.9152 - val_loss: 0.2129
Epoch 6/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 42s 239ms/step - binary_accuracy: 0.9190 - loss: 0.2033 - val_binary_accuracy: 0.9166 - val_loss: 0.2098
Epoch 7/10
174/174 ━━━━━━━━━━━━━━━━━━━━ 42s 239ms/step - binary_accuracy: 0.9221 - loss: 0.1994 - val_binary_accuracy: 0.9283 - va